In [8]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.callbacks import HumanApprovalCallbackHandler
from langchain_community.llms import Ollama
llm = Ollama(model="qwen3:1.7b")

def _should_check(serialized_obj: dict) -> bool:
    # 仅需要ShellTool的批准
    return serialized_obj.get("name") == "terminal"


def _approve(_input: str) -> bool:
    if _input == "echo 'Hello World'":
        return True
    msg = (
        "您是否批准以下输入？"
        "“Y”/“Yes”（不区分大小写）之外的任何内容都将被视为“否”。"
    )
    msg += "\n\n" + _input + "\n"
    resp = input(msg)
    return resp.lower() in ("yes", "y")

callbacks = [HumanApprovalCallbackHandler(should_check=_should_check, approve=_approve)]
 
tools = load_tools(["terminal","wikipedia", "llm-math" ], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

agent.run(
    "现在是2024年。康拉德·阿登纳多少年前成为德国总理?",
    callbacks=callbacks,
)
agent.run("print 'Hello World' in the terminal", callbacks=callbacks)


ValueError: terminal is a dangerous tool. You cannot use it without opting in by setting allow_dangerous_tools to True. Most tools have some inherit risk to them merely because they are allowed to interact with the "real world".Please refer to LangChain security guidelines to https://python.langchain.com/docs/security.Some tools have been designated as dangerous because they pose risk that is not intuitively obvious. For example, a tool that allows an agent to make requests to the web, can also be used to make requests to a server that is only accessible from the server hosting the code.Again, all tools carry some risk, and it's your responsibility to understand which tools you're using and the risks associated with them.